# Import all zillow rent indices

In [2]:
import pandas as pd

#filepath input is for if directories are different on each machine


#if csvs are in same directory as notebook, set filepath = './'
filepath = input('Enter file path where csv files are located on your machine\n')
multi = pd.read_csv(filepath + '/Zip_Zri_MultiFamilyResidenceRental.csv',dtype = {'RegionName':str})
homes = pd.read_csv(filepath + '/Zip_Zri_AllHomesPlusMultifamily.csv',dtype = {'RegionName':str})
single = pd.read_csv(filepath + '/Zip_Zri_SingleFamilyResidenceRental.csv',dtype = {'RegionName':str})




def zero_zips(df):
    '''Adds zeros to beginning of zip codes that were chopped off when of int dtype
    
    Args:
    
    df: name of the DataFrame'''
    return df['RegionName'].apply(lambda Z: '0'*(5-len(Z))+Z)

multi['RegionName'] = zero_zips(multi)
homes['RegionName'] = zero_zips(homes)
single['RegionName'] = zero_zips(single)

print(multi.shape)
print(single.shape)
print(homes.shape)

Enter file path where csv files are located on your machine
../../renycdsa7parkcapstonekickoff
(1861, 120)
(15836, 111)
(13181, 120)


In [3]:
multi = pd.melt(multi, id_vars =multi.columns[:7],
                value_vars = multi.columns[7:],
               var_name = 'Date',value_name = 'Rent')
multi['Date'] = pd.to_datetime(multi['Date'])
multi.rename(columns = {'RegionName':'Zipcode'}, inplace = True)



multi.sample(10)




,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,Date,Rent
31322,72709,33606,Tampa,FL,Tampa-St. Petersburg-Clearwater,Hillsborough County,1547,2012-01-01,1071.0
9160,62103,11417,New York,NY,New York-Newark-Jersey City,Queens County,1717,2011-01-01,1484.0
70338,73034,34210,Bradenton,FL,North Port-Sarasota-Bradenton,Manatee County,1482,2013-10-01,NaN
209298,71141,30458,Statesboro,GA,Statesboro,Bulloch County,867,2020-01-01,720.0
201538,65771,19103,Philadelphia,PA,Philadelphia-Camden-Wilmington,Philadelphia County,551,2019-09-01,1686.0
145447,70991,30263,Newnan,GA,Atlanta-Sandy Springs-Roswell,Coweta County,290,2017-03-01,NaN
4691,91834,77304,Conroe,TX,Houston-The Woodlands-Sugar Land,Montgomery County,970,2010-11-01,NaN
204048,93313,80237,Denver,CO,Denver-Aurora-Lakewood,Denver County,1200,2019-10-01,1484.0
97973,93290,80214,Lakewood,CO,Denver-Aurora-Lakewood,Jefferson County,1202,2015-01-01,979.0
199243,96545,91910,Chula Vista,CA,San Diego-Carlsbad,San Diego County,117,2019-08-01,NaN


## examining missingness by state

In [6]:
multi['State'].loc[multi['Metro'].isnull()].value_counts().reset_index().rename(
    columns = {'index':'State','State':'Null_count'})

,State,Null_count
0,UT,226
1,ME,113


In [7]:
na_metro = multi[['Metro','CountyName']].loc[multi.Metro.isnull()].copy()
na_metro['CountyName'].value_counts()

Sanpete County     226
Somerset County    113
Name: CountyName, dtype: int64

# All homes including multifamily

In [8]:
homes = pd.melt(homes,id_vars = homes.columns[:7], 
                 value_vars = homes.columns[7:], var_name = 'Date',
                 value_name='OldIndex')
# zillow['Date'] = pd.to_datetime(zillow['Date'])
homes.head(10)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,Date,OldIndex
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,2010-09,3031.0
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,2010-09,1790.0
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,2010-09,3269.0
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,2010-09,1547.0
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,2010-09,1922.0
5,91940,77449,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,6,2010-09,1168.0
6,61616,10002,New York,NY,New York-Newark-Jersey City,New York County,7,2010-09,3272.0
7,91733,77084,Houston,TX,Houston-The Woodlands-Sugar Land,Harris County,8,2010-09,932.0
8,93144,79936,El Paso,TX,El Paso,El Paso County,9,2010-09,1060.0
9,84640,60640,Chicago,IL,Chicago-Naperville-Elgin,Cook County,10,2010-09,1432.0


In [9]:
homes.rename(columns = {'RegionName':'ZipCode'},inplace=True)

In [10]:
homes['Date'] = pd.to_datetime(homes['Date'],format = '%Y-%m')
homes['Month'] = homes['Date'].dt.month
homes['Year'] = homes['Date'].dt.year
# homes.drop('Date',axis = 1, inplace = True)

In [11]:
homes.head()

,RegionID,ZipCode,City,State,Metro,CountyName,SizeRank,Date,OldIndex,Month,Year
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,2010-09-01,3031.0,9,2010
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,2010-09-01,1790.0,9,2010
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,2010-09-01,3269.0,9,2010
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,2010-09-01,1547.0,9,2010
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,2010-09-01,1922.0,9,2010


# Newest zillow rent index

In [13]:
zillow = pd.read_csv(filepath + '/Zip_ZORI_AllHomesPlusMultifamily_SSA.csv',dtype = {'RegionName':str})

In [14]:
zillow.rename(columns = {'RegionName':'ZipCode'},inplace=True)

In [15]:
zillow = pd.melt(zillow,id_vars = zillow.columns[:4], 
                 value_vars = zillow.columns[4:], var_name = 'Date',
                 value_name='NewIndex')
# zillow['Date'] = pd.to_datetime(zillow['Date'])
zillow.head(10)

,RegionID,ZipCode,SizeRank,MsaName,Date,NewIndex
0,61639,10025,1,"New York, NY",2014-01,3062.0
1,84654,60657,2,"Chicago, IL",2014-01,1544.0
2,61637,10023,3,"New York, NY",2014-01,3104.0
3,91982,77494,4,"Houston, TX",2014-01,1738.0
4,84616,60614,5,"Chicago, IL",2014-01,1702.0
5,91940,77449,6,"Houston, TX",2014-01,1299.0
6,61616,10002,7,"New York, NY",2014-01,2612.0
7,91733,77084,8,"Houston, TX",2014-01,1140.0
8,93144,79936,9,"El Paso, TX",2014-01,979.0
9,84640,60640,10,"Chicago, IL",2014-01,1278.0


In [16]:
zillow['Date'] = pd.to_datetime(zillow['Date'])

zillow['Month'] = zillow['Date'].dt.month
zillow['Year'] = zillow['Date'].dt.year

In [17]:
zillow[['City','State']] = zillow['MsaName'].str.split(',',expand = True)
zillow.drop('MsaName',axis = 1, inplace = True)

In [18]:
zillow.sample(15)

,RegionID,ZipCode,SizeRank,Date,NewIndex,Month,Year,City,State
193456,96531,91801,643,2020-03-01,2405.0,3,2020,Los Angeles-Long Beach-Anaheim,CA
182812,67720,23452,349,2019-11-01,1362.0,11,2019,Virginia Beach,VA
50617,93280,80203,1695,2015-08-01,1237.0,8,2015,Denver,CO
163969,58636,02122,4617,2019-03-01,2100.0,3,2019,Boston,MA
83890,73162,34668,605,2016-09-01,941.0,9,2016,Tampa,FL
140922,76953,44035,109,2018-07-01,684.0,7,2018,Cleveland,OH
97311,65795,19128,1253,2017-02-01,1414.0,2,2017,Philadelphia,PA
94393,69385,27526,710,2017-01-01,1379.0,1,2017,Raleigh,NC
92512,99580,98121,2015,2016-12-01,2311.0,12,2016,Seattle,WA
88367,58562,01950,4771,2016-10-01,1718.0,10,2016,Boston,MA


In [19]:
Nulls = zillow[['Date','City','State']].loc[zillow['NewIndex'].isnull()]
Nulls.head(20)

,Date,City,State
23,2014-01-01,Houston,TX
25,2014-01-01,Houston,TX
31,2014-01-01,Houston,TX
39,2014-01-01,Houston,TX
52,2014-01-01,Chicago,IL
61,2014-01-01,Houston,TX
63,2014-01-01,Houston,TX
65,2014-01-01,Austin,TX
66,2014-01-01,Baltimore,MD
86,2014-01-01,Sacramento,CA
